In [1]:
#!pip show tensorflow_datasets
!pip3 install emnist
!pip3 install tensorflow_datasets
!pip3 install opencv_python
!pip3 install matplotlib
!pip3 install chardet
!pip3 install pandas

In [1]:
# import emnist
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow.keras as keras
import sklearn
import random
from sklearn.decomposition import PCA 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import cv2
from tensorflow.keras.layers import Input, Lambda, Conv2D,Conv2DTranspose, MaxPooling2D, BatchNormalization, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K
%matplotlib inline
# tf.debugging.set_log_device_placement(False)

In [2]:
print(tf.config.list_physical_devices('GPU'))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
print(tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Default GPU Device: /device:GPU:0
True


In [3]:
(ds_train, ds_test), info = tfds.load('omniglot', split=['train', 'test'], with_info=True)

In [4]:
df_train = tfds.as_dataframe(ds_train, info)
df_test  = tfds.as_dataframe(ds_test, info)

In [5]:
train_images = np.stack(df_train['image'])
train_images.shape

(19280, 105, 105, 3)

In [6]:
df_train.columns

Index(['alphabet', 'alphabet_char_id', 'image', 'label'], dtype='object')

In [7]:
len(df_train['label'].unique())

964

In [8]:
# Hyperparameters

shots = 1
train_shots = 5
classes = 5
img_size = 56
batch_size = 25
num_mini_iterations = 1000

In [9]:
''' Data handling general functions '''

def separate_fewshot(test_images, test_labels, n=1):
    oneshot_data = []
    classify_data = []
    for label in np.unique(test_labels):
        for num in np.random.choice(np.where(test_labels == label)[0], n, False):
            oneshot_data.append(num)
    temp = set(oneshot_data)
    for i in range(len(test_labels)):
        if not i in temp: classify_data.append(i)
    oneshot_images = test_images[oneshot_data]
    oneshot_labels = test_labels[oneshot_data]
    classify_images = test_images[classify_data]
    classify_labels = test_labels[classify_data]
    return oneshot_images, oneshot_labels, classify_images, classify_labels

In [10]:
def resize_images(images, size, to_grayscale = True):
    resized_images = []
    for img in images:
        resized_image = cv2.resize(img, (size, size))
        if to_grayscale:
            resized_image= cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        resized_images.append(resized_image)
    return np.array(resized_images)

In [11]:
def parse_omniglot_dataframe(df, img_size = 56, reshape=True):
    images = resize_images(df['image'], img_size)
    if reshape:
        images = images.reshape(-1, img_size * img_size)
    else:
        images.reshape(-1, img_size, img_size)
        images = images[..., np.newaxis]
    labels = df['label'].to_numpy()
    data = {}
    for label in np.unique(labels):
        data[label] = images[np.where(labels == label)]
    return (data, labels)

In [12]:
def get_batch(data, labels, num_iterations=10, num_classes=classes, shots=shots, num_support=train_shots, img_size=img_size):
    labels = np.unique(labels)
    num_labels = labels.shape[0]
    np.random.shuffle(labels)
    
    x = np.zeros(shape=(num_classes * shots, img_size, img_size, 1))
    y = np.zeros(shape=(num_classes * num_support, img_size, img_size,1))
    x_l, y_l = np.zeros(num_classes * shots), np.zeros(num_classes * num_support)
    
    size = num_classes * num_classes * shots * num_support
    num_iterations = min(num_iterations, (num_labels + num_classes - 1) // num_classes)
    
    pairs = [[None for _ in range(size * num_iterations)], [None for _ in range(size * num_iterations)]]
    labels = [[None for _ in range(size * num_iterations)], [None for _ in range(size * num_iterations)]]
    target = np.zeros(size * num_iterations)
    
    cur = 0
    for t in range(0, num_iterations * num_classes, num_classes):
        classes = labels[t: t + num_classes]
        #classes = np.random.choice(labels, n_classes, False)        
        for i, c in enumerate(classes):
            ind = np.random.choice(np.arange(data[c].shape[0]), shots + num_support, False)
            x_ind = ind[:shots]
            #print(x_ind.shape, data[c][x_ind].shape, x[i * shots: (i+1) * shots])
            x[i * shots: (i+1) * shots, :][0] = data[c][x_ind]
            x_l[i * shots: (i+1) * shots] = c

            y_ind = ind[shots:]
            y[i * num_support: (i+1) * num_support, :] = data[c][y_ind]
            y_l[i * num_support: (i+1) * num_support] = c
            
        for i, xx in enumerate(x):
            if x_l[i] not in classes:
                continue
            for j, yy in enumerate(y):
                if y_l[j] not in classes:
                    continue
                pairs[0][cur] = xx
                pairs[1][cur] = yy
                labels[0][cur] = x_l[i]
                labels[1][cur] = y_l[i]
                target[cur] = ((x_l[i] == y_l[j]) and (x_l[i] != -1))
                cur += 1
    pairs[0] = np.array(pairs[0][:cur])
    pairs[1] = np.array(pairs[1][:cur])
    labels[0] = np.array(labels[0][:cur])
    labels[1] = np.array(labels[1][:cur])
    target = target[:cur]
    return pairs, target, labels

In [13]:
data, lbl = parse_omniglot_dataframe(df_test, reshape=False)
data[lbl[0]].shape
#get_train_batch(data, lbl)

([array([[[[255.],
           [255.],
           [255.],
           ...,
           [255.],
           [255.],
           [255.]],
  
          [[255.],
           [255.],
           [255.],
           ...,
           [255.],
           [255.],
           [255.]],
  
          [[255.],
           [255.],
           [255.],
           ...,
           [255.],
           [255.],
           [255.]],
  
          ...,
  
          [[255.],
           [255.],
           [255.],
           ...,
           [255.],
           [255.],
           [255.]],
  
          [[255.],
           [255.],
           [255.],
           ...,
           [255.],
           [255.],
           [255.]],
  
          [[255.],
           [255.],
           [255.],
           ...,
           [255.],
           [255.],
           [255.]]],
  
  
         [[[255.],
           [255.],
           [255.],
           ...,
           [255.],
           [255.],
           [255.]],
  
          [[255.],
           [255.],
  

In [14]:
def get_siamese_net_and_encoder(input_shape, code_size = 0):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    encoder = Sequential()
    encoder.add(Conv2D(64, (10, 10), input_shape=input_shape, activation='relu', kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2))
    encoder.add(Dropout(0.25))
    
    encoder.add(Conv2D(128, (7, 7), kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2))
    encoder.add(Dropout(0.25))
    
    encoder.add(Conv2D(128, (4, 4), kernel_regularizer='l2'))
    encoder.add(BatchNormalization())
    encoder.add(Activation('relu'))
    encoder.add(MaxPooling2D(pool_size=2))
    encoder.add(Dropout(0.25))
    #encoder.add(Conv2D(256, (4, 4), kernel_regularizer='l2'))
    
    encoder.add(Flatten())
    
    encoder.add(Dense(2048, activation='sigmoid', kernel_regularizer='l2'))
    
    left_emb = encoder(left_input)
    right_emb = encoder(right_input)
    
    L1_Layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_Dist = L1_Layer([left_emb,right_emb])
    OP = Dense(1, activation='sigmoid', kernel_regularizer='l2')(L1_Dist)
    
    siamese_net = Model(inputs=[left_input, right_input], outputs=OP)
    
    return siamese_net, encoder


In [15]:
siamese_net, encoder = get_siamese_net_and_encoder((img_size, img_size, 1))

siamese_net.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

siamese_net.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 56, 56, 1)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 56, 56, 1)]  0           []                               
                                                                                                  
 sequential (Sequential)        (None, 2048)         1722176     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 2048)         0           ['sequential[0][0]',         

In [16]:
def get_image_by_label(label, train_images, train_labels):
    return train_images[np.random.choice(np.where(train_labels == label)[0], 1, False)[0]]

def get_train_data(size, train_images, train_labels, img_size):
    targets = np.zeros((size,))
    targets[size // 2:] = 1
    pairs = [np.zeros((size, img_size, img_size)) for _ in range(2)]
    labels = np.unique(train_labels)
    for i in range(size):
        class1 = np.random.choice(labels, 1)[0]
        class2 = class1
        if i < size // 2:
            while class2 == class1:
                class2 = np.random.choice(labels, 1)[0]
        pairs[0][i] = get_image_by_label(class1, train_images, train_labels)
        pairs[1][i] = get_image_by_label(class2, train_images, train_labels)
    return pairs, targets

def visualize_siamese(siamese_net, train_images, train_labels, label1 = 1, label2=2):
    x = train_images[np.where(train_labels == label1)[0][:1]]
    y = train_images[np.where(train_labels == label2)[0][1:2]]
    d = siamese_net.predict([x, y], verbose=0)
    fig, axis = plt.subplots(1, 2, figsize=(10, 5))
    axis[0].axis("off")
    axis[1].axis("off")
    fig.suptitle(f"Output {d[0][0]}")
    axis[0].imshow(x[0], cmap='gray')
    axis[1].imshow(y[0], cmap='gray')
    plt.show()

In [17]:
train_data, train_labels = parse_omniglot_dataframe(df_train, img_size=img_size, reshape=False)
def train_one_shot():
    for i in range(1, num_mini_iterations + 1):
        print(f"Epoch {i}:")
        x, y, _ = get_batch(train_data, train_labels, shots=shots, num_classes=classes, num_support=train_shots, img_size=img_size)
        #print("shapes:", x[0].shape, x[1].shape, y.shape)
        #print("sum is:", np.sum(y), "should be:", n_iterations * n_classes * num_shots * n_support)
        loss = siamese_net.fit(x, y, batch_size=batch_size,)
        #if i % evaluateEvery == 0:
        #    print('Iteration', i, '- Loss:',loss[0],'- Acc:', round(loss[1], 2))
train_one_shot()

Epoch 1:
50/50 [==============================] - 14s 11ms/step - loss: 4.8252 - accuracy: 0.8360
Epoch 2:
50/50 [==============================] - 1s 10ms/step - loss: 0.8982 - accuracy: 0.8880
Epoch 3:
50/50 [==============================] - 1s 10ms/step - loss: 0.6697 - accuracy: 0.9328
Epoch 4:
50/50 [==============================] - 1s 10ms/step - loss: 0.5107 - accuracy: 0.9440
Epoch 5:
50/50 [==============================] - 1s 10ms/step - loss: 0.6112 - accuracy: 0.9016
Epoch 6:
50/50 [==============================] - 1s 10ms/step - loss: 0.5828 - accuracy: 0.9184
Epoch 7:
50/50 [==============================] - 1s 10ms/step - loss: 0.5558 - accuracy: 0.9312
Epoch 8:
50/50 [==============================] - 1s 10ms/step - loss: 0.5509 - accuracy: 0.9096
Epoch 9:
50/50 [==============================] - 1s 10ms/step - loss: 0.5201 - accuracy: 0.9424
Epoch 10:
50/50 [==============================] - 1s 10ms/step - loss: 0.4995 - accuracy: 0.9432
Epoch 11:
50/50 [===========

50/50 [==============================] - 1s 10ms/step - loss: 0.3745 - accuracy: 0.9768
Epoch 85:
50/50 [==============================] - 1s 11ms/step - loss: 0.4910 - accuracy: 0.9448
Epoch 86:
50/50 [==============================] - 1s 11ms/step - loss: 0.4442 - accuracy: 0.9544
Epoch 87:
50/50 [==============================] - 1s 11ms/step - loss: 0.5397 - accuracy: 0.9320
Epoch 88:
50/50 [==============================] - 1s 11ms/step - loss: 0.5420 - accuracy: 0.9296
Epoch 89:
50/50 [==============================] - 1s 11ms/step - loss: 0.4487 - accuracy: 0.9584
Epoch 90:
50/50 [==============================] - 1s 11ms/step - loss: 0.4509 - accuracy: 0.9520
Epoch 91:
50/50 [==============================] - 1s 10ms/step - loss: 0.5588 - accuracy: 0.9416
Epoch 92:
50/50 [==============================] - 1s 11ms/step - loss: 0.6025 - accuracy: 0.9336
Epoch 93:
50/50 [==============================] - 1s 11ms/step - loss: 0.5843 - accuracy: 0.9352
Epoch 94:
50/50 [=============

50/50 [==============================] - 1s 11ms/step - loss: 0.4698 - accuracy: 0.9632
Epoch 168:
50/50 [==============================] - 1s 11ms/step - loss: 0.4408 - accuracy: 0.9624
Epoch 169:
50/50 [==============================] - 1s 11ms/step - loss: 0.4647 - accuracy: 0.9552
Epoch 170:
50/50 [==============================] - 1s 11ms/step - loss: 0.4667 - accuracy: 0.9528
Epoch 171:
50/50 [==============================] - 1s 11ms/step - loss: 0.3673 - accuracy: 0.9672
Epoch 172:
50/50 [==============================] - 1s 11ms/step - loss: 0.4315 - accuracy: 0.9520
Epoch 173:
50/50 [==============================] - 1s 11ms/step - loss: 0.4744 - accuracy: 0.9536
Epoch 174:
50/50 [==============================] - 1s 11ms/step - loss: 0.4094 - accuracy: 0.9520
Epoch 175:
50/50 [==============================] - 1s 11ms/step - loss: 0.4005 - accuracy: 0.9664
Epoch 176:
50/50 [==============================] - 1s 11ms/step - loss: 0.3568 - accuracy: 0.9688
Epoch 177:
50/50 [===

50/50 [==============================] - 1s 11ms/step - loss: 0.4679 - accuracy: 0.9576
Epoch 250:
50/50 [==============================] - 1s 11ms/step - loss: 0.4144 - accuracy: 0.9680
Epoch 251:
50/50 [==============================] - 1s 11ms/step - loss: 0.4556 - accuracy: 0.9664
Epoch 252:
50/50 [==============================] - 1s 11ms/step - loss: 0.5059 - accuracy: 0.9608
Epoch 253:
50/50 [==============================] - 1s 11ms/step - loss: 0.5422 - accuracy: 0.9416
Epoch 254:
50/50 [==============================] - 1s 11ms/step - loss: 0.3843 - accuracy: 0.9760
Epoch 255:
50/50 [==============================] - 1s 11ms/step - loss: 0.3564 - accuracy: 0.9792
Epoch 256:
50/50 [==============================] - 1s 11ms/step - loss: 0.5464 - accuracy: 0.9432
Epoch 257:
50/50 [==============================] - 1s 11ms/step - loss: 0.4560 - accuracy: 0.9600
Epoch 258:
50/50 [==============================] - 1s 11ms/step - loss: 0.5500 - accuracy: 0.9560
Epoch 259:
50/50 [===

50/50 [==============================] - 1s 11ms/step - loss: 0.3642 - accuracy: 0.9824
Epoch 332:
50/50 [==============================] - 1s 11ms/step - loss: 0.3681 - accuracy: 0.9832
Epoch 333:
50/50 [==============================] - 1s 11ms/step - loss: 0.3511 - accuracy: 0.9768
Epoch 334:
50/50 [==============================] - 1s 11ms/step - loss: 0.3710 - accuracy: 0.9664
Epoch 335:
50/50 [==============================] - 1s 11ms/step - loss: 0.4293 - accuracy: 0.9608
Epoch 336:
50/50 [==============================] - 1s 11ms/step - loss: 0.5172 - accuracy: 0.9552
Epoch 337:
50/50 [==============================] - 1s 11ms/step - loss: 0.3831 - accuracy: 0.9728
Epoch 338:
50/50 [==============================] - 1s 11ms/step - loss: 0.4045 - accuracy: 0.9648
Epoch 339:
50/50 [==============================] - 1s 11ms/step - loss: 0.3074 - accuracy: 0.9808
Epoch 340:
50/50 [==============================] - 1s 11ms/step - loss: 0.4212 - accuracy: 0.9600
Epoch 341:
50/50 [===

...

50/50 [==============================] - 1s 11ms/step - loss: 0.5008 - accuracy: 0.9608
Epoch 660:
50/50 [==============================] - 1s 11ms/step - loss: 0.4518 - accuracy: 0.9696
Epoch 661:
50/50 [==============================] - 1s 11ms/step - loss: 0.4894 - accuracy: 0.9632
Epoch 662:
50/50 [==============================] - 1s 11ms/step - loss: 0.4064 - accuracy: 0.9752
Epoch 663:
50/50 [==============================] - 1s 11ms/step - loss: 0.3652 - accuracy: 0.9760
Epoch 664:
50/50 [==============================] - 1s 11ms/step - loss: 0.4513 - accuracy: 0.9688
Epoch 665:
50/50 [==============================] - 1s 11ms/step - loss: 0.5577 - accuracy: 0.9536
Epoch 666:
50/50 [==============================] - 1s 11ms/step - loss: 0.3550 - accuracy: 0.9832
Epoch 667:
50/50 [==============================] - 1s 11ms/step - loss: 0.4237 - accuracy: 0.9768
Epoch 668:
50/50 [==============================] - 1s 11ms/step - loss: 0.4821 - accuracy: 0.9704
Epoch 669:
50/50 [===

50/50 [==============================] - 1s 11ms/step - loss: 0.4961 - accuracy: 0.9624
Epoch 742:
50/50 [==============================] - 1s 11ms/step - loss: 0.4024 - accuracy: 0.9768
Epoch 743:
50/50 [==============================] - 1s 11ms/step - loss: 0.4527 - accuracy: 0.9704
Epoch 744:
50/50 [==============================] - 1s 11ms/step - loss: 0.3949 - accuracy: 0.9736
Epoch 745:
50/50 [==============================] - 1s 11ms/step - loss: 0.3954 - accuracy: 0.9704
Epoch 746:
50/50 [==============================] - 1s 11ms/step - loss: 0.2902 - accuracy: 0.9896
Epoch 747:
50/50 [==============================] - 1s 11ms/step - loss: 0.3953 - accuracy: 0.9792
Epoch 748:
50/50 [==============================] - 1s 11ms/step - loss: 0.3882 - accuracy: 0.9736
Epoch 749:
50/50 [==============================] - 1s 11ms/step - loss: 0.4167 - accuracy: 0.9672
Epoch 750:
50/50 [==============================] - 1s 11ms/step - loss: 0.4372 - accuracy: 0.9720
Epoch 751:
50/50 [===

50/50 [==============================] - 1s 11ms/step - loss: 0.5060 - accuracy: 0.9560
Epoch 824:
50/50 [==============================] - 1s 11ms/step - loss: 0.5425 - accuracy: 0.9584
Epoch 825:
50/50 [==============================] - 1s 11ms/step - loss: 0.4420 - accuracy: 0.9680
Epoch 826:
50/50 [==============================] - 1s 11ms/step - loss: 0.3770 - accuracy: 0.9800
Epoch 827:
50/50 [==============================] - 1s 11ms/step - loss: 0.4163 - accuracy: 0.9744
Epoch 828:
50/50 [==============================] - 1s 11ms/step - loss: 0.5619 - accuracy: 0.9496
Epoch 829:
50/50 [==============================] - 1s 11ms/step - loss: 0.4862 - accuracy: 0.9688
Epoch 830:
50/50 [==============================] - 1s 11ms/step - loss: 0.4329 - accuracy: 0.9720
Epoch 831:
50/50 [==============================] - 1s 11ms/step - loss: 0.4003 - accuracy: 0.9680
Epoch 832:
50/50 [==============================] - 1s 11ms/step - loss: 0.3391 - accuracy: 0.9816
Epoch 833:
50/50 [===

50/50 [==============================] - 1s 11ms/step - loss: 0.3869 - accuracy: 0.9808
Epoch 906:
50/50 [==============================] - 1s 11ms/step - loss: 0.3987 - accuracy: 0.9712
Epoch 907:
50/50 [==============================] - 1s 11ms/step - loss: 0.5278 - accuracy: 0.9584
Epoch 908:
50/50 [==============================] - 1s 11ms/step - loss: 0.5316 - accuracy: 0.9632
Epoch 909:
50/50 [==============================] - 1s 11ms/step - loss: 0.5170 - accuracy: 0.9632
Epoch 910:
50/50 [==============================] - 1s 11ms/step - loss: 0.3643 - accuracy: 0.9816
Epoch 911:
50/50 [==============================] - 1s 11ms/step - loss: 0.3409 - accuracy: 0.9800
Epoch 912:
50/50 [==============================] - 1s 11ms/step - loss: 0.3622 - accuracy: 0.9792
Epoch 913:
50/50 [==============================] - 1s 11ms/step - loss: 0.5247 - accuracy: 0.9512
Epoch 914:
50/50 [==============================] - 1s 11ms/step - loss: 0.4368 - accuracy: 0.9712
Epoch 915:
50/50 [===

50/50 [==============================] - 1s 11ms/step - loss: 0.3562 - accuracy: 0.9744
Epoch 988:
50/50 [==============================] - 1s 11ms/step - loss: 0.4010 - accuracy: 0.9624
Epoch 989:
50/50 [==============================] - 1s 11ms/step - loss: 0.4518 - accuracy: 0.9632
Epoch 990:
50/50 [==============================] - 1s 11ms/step - loss: 0.3236 - accuracy: 0.9856
Epoch 991:
50/50 [==============================] - 1s 11ms/step - loss: 0.4871 - accuracy: 0.9696
Epoch 992:
50/50 [==============================] - 1s 11ms/step - loss: 0.5927 - accuracy: 0.9544
Epoch 993:
50/50 [==============================] - 1s 11ms/step - loss: 0.5307 - accuracy: 0.9584
Epoch 994:
50/50 [==============================] - 1s 11ms/step - loss: 0.4630 - accuracy: 0.9744
Epoch 995:
50/50 [==============================] - 1s 11ms/step - loss: 0.4041 - accuracy: 0.9688
Epoch 996:
50/50 [==============================] - 1s 11ms/step - loss: 0.4683 - accuracy: 0.9616
Epoch 997:
50/50 [===

In [19]:
# visualize_siamese(siamese_net, train_images, train_labels, label1=63, label2=62)

In [20]:
matches = 0
total = 0
def get_mode(l):
    d = {}
    mode = None
    count = 0
    for i in l:
        if i in d:
            d[i] += 1
        else:
            d[i] = 1
        if d[i] > count:
            mode = i
    return mode

test_data, test_labels = parse_omniglot_dataframe(df_test, img_size=img_size, reshape=False)
test_times = 100
unique_labels = np.unique(test_labels)
for _ in range(test_times):
    samples, _, labels = get_train_batch()
    t = siamese_net.predict(samples, verbose=0)
    
    


# for alphabet in np.unique(df_test['alphabet']):
#     ind_alphabet = np.where(df_test['alphabet'] == alphabet)[0]
#     labels = test_labels[ind_alphabet]
#     images = test_images[ind_alphabet]
#     os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=num_shots)
    
#     l_os = os_img
#     l_clas = clas_img
#     print(len(l_os), len(l_clas))
#     x, y = [], []
#     for i, c_i in enumerate(l_clas):
#         x += [c_i.copy() for _ in range(len(l_os))]
#         y = [*y, *l_os]
#     x = np.array(x)
#     y = np.array(y)
#     print(x.shape, y.shape)
#     print(f"Projecting alphabet {alphabet}")
#     t = siamese_net.predict([x, y], verbose=0)
#     print("Testing")
#     for i in range(len(clas_img)):
#         arr = t[i * len(os_img): (i + 1) * len(os_img)].reshape(-1)
#         ind = np.argsort(arr)[-num_shots:]
#         ind = get_mode(ind)
#         pred = os_label[ind]
#         #print(t.shape, ind, pred, clas_label[i])
#         matches += np.sum(pred == clas_label[i])
#         total += 1

# print("Accuracy: ", matches/total)
# print("======= NL Autoencoder method: Finished =======")

NameError: name 'clas_label' is not defined

In [173]:
matches = total = 0
print(f"Trying the encoder with {num_shots} shots")
for alphabet in np.unique(df_test['alphabet']):
    ind_alphabet = np.where(df_test['alphabet'] == alphabet)[0]
    labels = test_labels[ind_alphabet]
    images = test_images[ind_alphabet]
    os_img, os_label, clas_img, clas_label = separate_fewshot(images, labels, n=num_shots)
    
    os_img = encoder.predict(os_img)
    clas_img = encoder.predict(clas_img)
    
    neigh = KNeighborsClassifier(n_neighbors = num_shots)
    neigh.fit(os_img, os_label)
    
    pred = neigh.predict(clas_img)
    matches += np.sum(pred == clas_label)
    total += len(clas_label)
print(matches/total)

Trying the encoder with 1 shots
16/16 [==============================] - 0s 2ms/step
0.14910949604664164


In [ ]:
## total = matches = 0
os_img, os_label, clas_img, clas_label = separate_fewshot(test_images, test_labels, n=1)
os_img = encoder.predict(os_img)
clas_img = encoder.predict(clas_img)

#if verbose: print("Learning oneshot ...")
#nn = min(train, 5)
neigh = KNeighborsClassifier(n_neighbors = 1)
neigh.fit(os_img, os_label)

#if verbose: print("Predicting ...")
pred = neigh.predict(clas_img)

matches += np.sum(pred == clas_label)
total += len(clas_label)
print(matches / total)